We define the different packages necessary to execute our code

In [29]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
import numpy as np

from SkinDetector import SkinDetector
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras
from SkinDetector import SkinDetector

from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Dropout
from keras.layers import Flatten, BatchNormalization
import tensorflow as tf

Here we initialize the "SkinDetector", our previous phase code, that binarizes images detecting skin pixels and setting them to white while the background is set to black. We then process the binarized images and set them to the same size, 480x640, this is necessary because if we do not resize the images, the model will not train them. 
We resize them by creating a new black image, the same colour as our background, and them inserting the white pixels corresponding to the detected skin. We do the same with the test images aswell.
We process the labels too, by setting them to categorical so the model will work correctly. 
(The code executes satisfactorily and the warnings are not a problem)

In [30]:
sd = SkinDetector()
train = sd.segment_dataset(sd.TR_DATA)
train_data = []
for i in range(0,60):
    result = np.zeros((480,640))
    result[:train[i].shape[0],:train[i].shape[1]] = train[i]
    train_data.append(result)
train_data = np.asarray(train_data)
train_labels = np.asarray(sd.TR_LABEL)

test = sd.segment_dataset(sd.VD_DATA)
test_data = []
for i in range(0,test.shape[0]):
    result = np.zeros((480,640))
    result[:test[i].shape[0],:test[i].shape[1]] = test[i]
    test_data.append(result)

test_data = np.asarray(test_data)
test_labels = np.asarray(sd.VD_LABEL)

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

/Users/oriolnarvaez/opt/anaconda3/lib/python3.7/site-packages/skimage/color/colorconv.py:271: RuntimeWarning: divide by zero encountered in true_divide
  out[idx, 0] = (arr[idx, 1] - arr[idx, 2]) / delta[idx]
/Users/oriolnarvaez/opt/anaconda3/lib/python3.7/site-packages/skimage/color/colorconv.py:279: RuntimeWarning: divide by zero encountered in true_divide
  out[idx, 0] = 4. + (arr[idx, 0] - arr[idx, 1]) / delta[idx]
/Users/oriolnarvaez/opt/anaconda3/lib/python3.7/site-packages/skimage/color/colorconv.py:265: RuntimeWarning: divide by zero encountered in true_divide
  out_s = delta / out_v
/Users/oriolnarvaez/opt/anaconda3/lib/python3.7/site-packages/skimage/color/colorconv.py:275: RuntimeWarning: divide by zero encountered in true_divide
  out[idx, 0] = 2. + (arr[idx, 2] - arr[idx, 0]) / delta[idx]


Here we define our model, as we can see this is a Convolutional based model, with just one convolutional layer, some max pooling and two linear layers at the end to classify into the 6 possible classes (0 fingers to 5 fingers).
The first convolutional layer is taking 4 arguments, the first is the number of filters, 32, the second one is the size of the filters, 3x3, the the size of the images, in our case 480x640 and the "1" to indicate they are binary images. Lastly the activation fucntion, here "relu" stands for a recrtifier function.
Now we perform a pooling operation on the resultant feature maps we get after the convolution is done on an image. The primary aim of a pooling layers is to reduce the size of the images as much as possible, we are trying to reduce the total number of nodes for the upcoming layers. To do this pooling we take a 2x2 matrix, to minimize pixel loss and get a precise region where the feature are located.
The as we can see, we flatten hte pooled images, this is pretty self-explanatory, we just transform the 2D array (pooled images) and converting them to a one dimensional single vector.
The next one we can refer to it as a "hidden layer", just because is between the input and output layers, this is a dense layer which means is fully connected. The parameter 'units' is where we define the number of nodes that should be present in this hidden layers, the number is not optimal but a close approximation to the best one and it is a commin practice to use a power of 2, in this case 128. The activation function will be a recrifier function, the same as the convolutional layer.
Now it is time to initialise our output layer, which should contain 6 nodes, because we have 6 classes. We will be using a sigmoid activation function.

In [31]:
classifier = Sequential()
classifier.add(Conv2D(32, (3, 3),padding='same',input_shape = (480, 640, 1), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(Flatten())
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 6, activation = 'sigmoid'))

We now compile the model. We have to say that even though it says that is using a "binary_crossentropy" loss, this is not to worry, because we have 6 classes, so no binary, but the data is transformed in a way that this is the adequate loss function. If we use the "categorical crossentropy" we obtain bad results, because the data is transformed to work with the other one. We just wanted to say this is not a mistake, this is "planned". 
We can also chose the optimizer, in this case "adam" because we tried different ones but they are mostly the same.
Then in "metrics" we chose to display the training accuracy and training recall.

In [32]:
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy',tf.keras.metrics.Recall()])

Here we make sure that the labels are correct, and categorically coded.

In [33]:
train_labels.shape

(60, 6)

Here we transform the data to comply with the expected input of the model, the "reshape" seems redundant, but we really resize the images earlier, this is just to sort the dimensions so the model will work as expected.

In [34]:
rows, cols = 480, 640
train_data = train_data.reshape(train_data.shape[0], rows, cols, 1)
test_data = test_data.reshape(test_data.shape[0], rows, cols, 1)

train_data = train_data.astype('float32')
test_data = test_data.astype('float32')

We make sure the reshape work correctly.

In [35]:
train_data.shape

(60, 480, 640, 1)

Then we train the model, with 20 epochs. ("verbose = 1" is just to display the results as we are training)
Here we want to clarify that we used all of the training images to actually train, because we tried other training methods using a percentatge of images as a validation, usually 10% or 20%, but because we have so few samples, only 60, this reduction on number of training images really affected the results. So is best to train will all the images and not with a validation set. We thought this will lead to overfit, but as the test results can demonstrate, this model is not overfitted.

In [39]:
history = classifier.fit(train_data, train_labels, epochs=20, verbose=1)

Epoch 1/20
60/60 [==============================] - 209s 3s/step - loss: 7.2783 - accuracy: 0.5944 - recall_2: 0.3833
Epoch 2/20
60/60 [==============================] - 164s 3s/step - loss: 3.8268 - accuracy: 0.8139 - recall_2: 0.3522
Epoch 3/20
60/60 [==============================] - 146s 2s/step - loss: 1.1112 - accuracy: 0.9139 - recall_2: 0.5189
Epoch 4/20
60/60 [==============================] - 176s 3s/step - loss: 0.5631 - accuracy: 0.9361 - recall_2: 0.6008
Epoch 5/20
60/60 [==============================] - 276s 5s/step - loss: 0.2075 - accuracy: 0.9722 - recall_2: 0.6637
Epoch 6/20
60/60 [==============================] - 291s 5s/step - loss: 0.1867 - accuracy: 0.9694 - recall_2: 0.7065
Epoch 7/20
60/60 [==============================] - 224s 4s/step - loss: 0.0540 - accuracy: 0.9889 - recall_2: 0.7374
Epoch 8/20
60/60 [==============================] - 178s 3s/step - loss: 0.0312 - accuracy: 0.9944 - recall_2: 0.7691
Epoch 9/20
60/60 [==============================] - 144s

Lastly, with the model trained, is time to evaluate it. We do so with the ".evaluate" function of the own model with the "test_dataset" and we get test loss, accuracy and recall.

In [43]:
print('\n# Evaluate on test data')
results = classifier.evaluate(test_data, test_labels)
print('test loss, test acc, test recall:', results)


# Evaluate on test data
45/45 [==============================] - 8s 174ms/step
test loss, test acc, test recall: [4.735368506113688, 0.7333333492279053, 0.8640578985214233]


Here we compute the F1-score as defined in the given project description.

In [42]:
(2*(results[1]*results[2]))/(results[1]+results[2])

0.8030356138432837